# Doc de la bibliothèque python construite pour aider à l'exploration lors du datasprint PORTIC 2021

## Objectif de la bibliothèque

- proposer deux clients pour récupérer les données PORTIC et de TOFLIT18, faciles à manipuler et avec une API la plus unifiée possible
- fournir des utils qui offrent des formatages des données adaptés (récupération de tableaux, construction de réseaux), et qui facilitent le croisement des données PORTIC / TOFLIT18

## Information sur les données

Concernant les clients web pour l'accès aux données, vous pourrez également accéder par vos propres moyens à partir des sources existantes :
   

- Voir la doc de l'API PORTIC : https://gitlab.huma-num.fr/portic/porticapi/-/tree/master/porticapi]
- Voir le répertoire des données toflit18 au format csv : https://github.com/medialab/toflit18_data)
- Voir le code de l'API toflit18 : https://github.com/medialab/toflit18)

## Installer la bibliothèque


La bibliothèque est installée localement par le script `install.sh`, mais vous pouvez également l'installer depuis une copie du répertoire github via `pip install -e lib)`. Ne pas oublier de la réinstaller après un git pull dans le cas où elle aurait changé.
    
## Sommaire des fonctions proposées
 
#### Fonctions principales (donnent accès aux données de base directement utilisables pour l'étude) :

Portic:

- [get_pointcalls](#portic_client.get_pointcalls(year=1787,-kwargs\)) : récupération de pointcalls (filtres disponibles : année, port d'enregistrement, entrée ou sortie dans le port, type de navire, activité déclarée (pêche, transport de marchandises ...), ...)

Toflit :

- [get_flows](#toflit_client.get_flows(start_year=None,-end_year=None,-year=None,-params=None,-kwargs\)) : récupération de flows (1 flow = import / export d'une marchandise ; infos disponibles : import ou export, produit commercé, partenaire commercial, bureau des Fermes ou a été enregistré le flux ...)



#### Fonctions secondaires (plutôt pour aider à paramétrer les fonctions principales, nourrir des fonctions de visualisation, ...)

Utils : 

- [nest_toflit18_flow](#nest_toflit18_flow(flow\)) : organise un flux toflit18 en un ensemble de clés thématiques
- [nest_portic_pointcall](#nest_portic_pointcall(pointcall\)) : organise un pointcall portic en un ensemble de clés thématiques
- [build_cooccurence_graph](#build_cooccurence_graph(data,-key_1,-key_2,-**kwargs\)) : renvoie un graphe networkx de coocurrence entre des dicts (besoin de fournir 2 clés)
- [get_online_csv](#get_online_csv(url\)) : récupère le contenu d'un csv en ligne (style google spreadsheet)
- [get_pointcalls_commodity_purposes_as_toflit_product](#get_pointcalls_commodity_purposes_as_toflit_product(pointcalls,-product_classification="product_simplification"\)) : correspondance de produits Portic / Toflit sur des pointcalls données en argument 
- [get_pointcalls_port_as_toflit_partner](#get_pointcalls_port_as_toflit_partner(pointcalls,-product_classification="partner_simplification"\))  : correspondance de ports d'enregistrement des congés Portic / partenaires Toflit sur des pointcalls données en argument 
- [get_pointcalls_homeport_as_toflit_partner](#get_pointcalls_homeport_as_toflit_partner(pointcalls,-product_classification="partner_simplification"\)) : correspondance de ports d'appartenance des navires Portic / partenaires Toflit sur des pointcalls données en argument 

Portic:

- [get_flows](#portic_client.get_flows(kwargs\)) : récupération de flux (1 flux portic = 1 trajet intermédiaire (escale), souvent décrit par plusieurs pointcalls)
- [get_travels](#portic_client.get_travels(kwargs\)) : récupération de travels (1 travel = ensemble de flux/escales qui s'enchainent ; infos disponibles : point de départ, d'arrivée, distance parcourue, infos sur le navire ...)
- [get_fieldnames](#portic_client.get_fieldnames(\)) : récupère les noms des variables des données et leur description
- [get_departures_details](#portic_client.get_departures_details(lon=None,-lat=None,-radius=None,-kwargs\)) : récupère tous les ports aux alentours d'un point géographique donné
- [get_destinations_aggregated](#portic_client.get_destinations_aggregated(lon=None,-lat=None,-radius=None,-kwargs\)) : retourne une aggrégation des amirautés de départ de voyages qui arrivent dans des points situés dans le voisinage (voir paramètre radius) du point requêté.
- [get_ports](#portic_client.get_ports(srid=None\)) : description de ports et de leur localisation géographique

Toflit :

- [get_flows_per_year](#toflit_client.get_flows_per_year(type,-kwargs\)) : récupère les flux par année par direction ou par type de source
- [get_locations](#toflit_client.get_locations(classification='partner_orthographic',-kwargs\)) : récupère le réseau des lieux (directions et partenaires) et le montant de leurs échanges 
- [get_time_series](#toflit_client.get_time_series(kwargs\)) : récupère des séries temporelles à propos des flux de marchandises
- [get_customs_regions](#toflit_client.get_customs_regions(\)) : récupère toutes les direction des Fermes de la base Toflit18
- [get_product_classifications](#toflit_client.get_product_classifications(\)) : récupère des informations sur une classification de produit toflit (et optionellement sur ses classifications filles)
- [get_partner_classifications](#toflit_client.get_partner_classifications(\)) : récupère des informations sur une classification de partenaires toflit (et optionellement sur ses classifications filles)
- [get_classification_groups](#toflit_client.get_classification_groups(classification="product_simplification",-kwargs\)) : récupère l'ensemble des catégories associées à une certaine classification (donnée en paramètre)
- [get_sources_types](#toflit_client.get_sources_types(\)) : récupère les types de sources disponibles


### Principe généraux d'utilisation du client

Les fonctions Portic et Toflit sont appelées respectivement depuis portic_client et toflit_client.
Les fonctions utils sont appelables directement.

Si des paramètres spécifiques peuvent être utilisés, ils sont décrit dans la doc spécifique à chaque fonction.

# Imports et bootstrapping des clients

In [1]:
import networkx as nx
import requests
import csv
import json

# liste des imports possibles
from poitousprint import Portic
from poitousprint import Toflit
from poitousprint import nest_toflit18_flow
from poitousprint import nest_portic_pointcall
from poitousprint import build_cooccurence_graph
from poitousprint import get_pointcalls_commodity_purposes_as_toflit_product
from poitousprint import get_online_csv
from poitousprint import get_pointcalls_port_as_toflit_partner
from poitousprint import get_pointcalls_homeport_as_toflit_partner

portic_client = Portic()
toflit_client = Toflit()

# Méthodes pour l'accès à PORTIC
pouvant être appelées depuis `portic_client` 

Paramètres généraux pouvant être appliqués à (presque) toutes ces méthodes :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
|params|liste de chaînes| ('all', ... tous les noms de params) | all | attributs à intégrer dans la réponse |
|format|chaîne| ('json','csv')| json | format de la réponse|
|shortenfields| booléen | (true, false) | false | permet de raccourcir les noms des attributs et donc d'alléger la taille du JSON téléchargé.|
|both_to| booléen | (true,false) | false | permet de récupérer les données concernant l'arrivée d'un voyage en plus des données du départ (données par défaut). |
| date | string format YYYY | (...années) | 1787 | pour filtrer les données sur une année donnée. L'année de la date d'arrivée ou de la date de départ doit commencer par ces 4 digits : 1787 ou  1789 par exemple. Exemple : http://data.portic.fr/api/pointcalls/?format=json&date=1789 |
| zipped | booléen | (true, false) | false | ... |

# portic_client.get_pointcalls(year=1787, kwargs)


Synopsis: récupère une liste de pointcalls selon les filtres et les paramètres spécifés.

---

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
| start_year | <int/string> | | | année de début |
| end_year | <int/string> | | | année de fin |
| year | <int/string> | | | année exacte (l'emporte sur début/fin) |

\+ tous les noms de propriétés des pointcalls pouvant être donnés en paramètres pour faire office de filtres (accepte une valeur unique, ou une liste de valeurs possibles pour que l'objet filtré soit valide - ex `pointcall_admirality='La Rochelle'`)


### Exemple : je veux tous les pointcalls concernant le datasprint

In [6]:
from poitousprint import Portic
portic_client = Portic()


pointcalls_datasprint = portic_client.get_pointcalls(
 source_subset = 'Poitou_1789',
)

""" équivalent à 

pointcalls_datasprint = portic_client.get_pointcalls(
 year = 1789,
 pointcall_admiralty = ['La Rochelle', 'Marennes', 'Sables-d’Olonne']
)
"""

pointcall = pointcalls_datasprint[0] 
# util pour organiser un pointcall en ensembles de clés thématiques
nest_portic_pointcall(pointcall)

test commodity_purpose val: None
test commodity_purpose2 val: None
test commodity_purpose3 val: None
test commodity_purpose4 val: None


{'source': {'source_doc_id': '00341028',
  'source_text': 'ANF, G5-62/5681',
  'source_suite': 'G5',
  'source_component': 'ANF, G5-62',
  'source_number': 5681.0,
  'source_other': None,
  'source_main_port_uhgs_id': 'A0171758',
  'source_main_port_toponyme': 'Charente',
  'source_subset': 'Poitou_1789',
  'navigo_status': 'FC-RF',
  'source_1787_available': True,
  'source_1789_available': True,
  'record_id': '00350929'},
 'pointcall_dates': {'pointcall_in_date': '1789>12>23',
  'pointcall_out_date': None,
  'date_fixed': None,
  'pointcall_outdate_uncertainity': -1,
  'outdate_fixed': None,
  'indate_fixed': None,
  'pointcall_in_date2': '1789=12=23',
  'pointcall_out_date2': None},
 'pointcall_localization': {'pkid': 94374,
  'pointcall': 'Marennes',
  'pointcall_uhgs_id': 'A0136930',
  'toponyme_fr': 'Marennes',
  'toponyme_en': 'Marennes',
  'latitude': '45.816667',
  'longitude': '-1.116667',
  'pointcall_admiralty': 'Marennes',
  'pointcall_province': 'Saintonge',
  'pointcall

## portic_client.get_fieldnames(API='any')


Synopsis:
Récupère les noms des variables Portic et leur description (pratique quand on ne sait plus ce qu'une colonne/propriété signifie)

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
| API | <string> | 'pointcalls' | 'travels' | 'any' | 'any' | type de fieldnames à récupérer |


---

### Exemple :

In [3]:
from poitousprint import Portic
portic_client = Portic()


portic_client.get_fieldnames()[0:1]

[{'api': 'None',
  'name': 'travel_id',
  'shortname': 't001',
  'type': 'text',
  'description': 'Identifiant du trajet'}]

## portic_client.get_flows(kwargs)

Synopsis:

Retourne une liste de flux, c'est-à-dire de voyages liés à des ports spécifiques, soit en y entrant (direction "in"), soit en en sortant (direction "out"), soit en ayant navigué autour (direction "in-out")

**Attention, les flux sont reconstruits d'après plusieurs pointcalls en fonction de leur incertitude, et sont peu complets pour les données du datasprint.** Il vaut mieux raisonner avec les pointcalls par défaut.

---

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
| ports | list(string) | UHGS_id | 4326 | liste des ids de ports à filtrer (séparés par des virgules)

\+ tous les noms de propriétés des flows portic pouvant être donnés en paramètres pour faire office de filtres (accepte une valeur unique, ou une liste de valeurs possibles pour que l'objet filtré soit valide)



### Exemple : tous les flux passant par Bordeaux (A0180923) ou Boulogne sur Mer (A0152606) ; on filtre les résultats sur les localisations de départ et d'arrivée, et la distance entre ces 2 points

In [6]:
from poitousprint import Portic
portic_client = Portic()


flows = portic_client.get_flows(
    ports = ['A0180923', 'A0152606'],
    params =['distance_dep_dest', 'departure', 'departure_latitude', 'departure_longitude', 'destination', 'destination_latitude', 'destination_longitude']
)
flows[0:1]

[{'distance_dep_dest': 0.0,
  'departure': 'Aber Ildut',
  'departure_latitude': '48.466667',
  'departure_longitude': '-4.75',
  'destination': 'Aber Ildut',
  'destination_latitude': '48.466667',
  'destination_longitude': '-4.75'}]

## portic_client.get_travels(kwargs)


Synopsis:

Récupère les données de trajectoires calculées.

#### possibilité de filtrer à une valeur ou une liste de valeurs par attribut
---

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|


### Exemple : tous les voyages au départ d'un port rattaché à la direction des Fermes de La Rochelle, dont le port d'attache du navire est Dieppe et Noirmoutier, et son pavillon français, ...

In [5]:
from poitousprint import Portic
portic_client = Portic()


portic_client.get_travels(
    departure_action='Out',
    departure_ferme_direction='La Rochelle',
    homeport_toponyme_fr=['Dieppe', 'Noirmoutier'],
    ship_flag_standardized_en='French'
)[0:1]

[{'travel_id': '0003811N- 21',
  'distance_dep_dest': 85.603,
  'distance_homeport_dep': 0.0,
  'departure': 'Noirmoutier',
  'departure_fr': 'Noirmoutier',
  'departure_en': 'Noirmoutier',
  'departure_uhgs_id': 'A0136403',
  'departure_latitude': '47',
  'departure_longitude': '-2.25',
  'departure_admiralty': 'Sables-d’Olonne',
  'departure_province': 'Poitou',
  'departure_states': '[{"1749-1815" : "France"}]',
  'departure_substates': None,
  'departure_state_1789_fr': 'France',
  'departure_substate_1789_fr': None,
  'departure_state_1789_en': 'France',
  'departure_substate_1789_en': None,
  'departure_ferme_direction': 'La Rochelle',
  'departure_ferme_bureau': "Sables d'Olonne",
  'departure_ferme_bureau_uncertainty': 0.0,
  'departure_partner_balance_1789': 'Petites Iles',
  'departure_partner_balance_supp_1789': 'France',
  'departure_partner_balance_1789_uncertainty': -1.0,
  'departure_partner_balance_supp_1789_uncertainty': 0.0,
  'departure_shiparea': 'ACE-YEUX',
  'depa

## portic_client.get_departures_details(lon=None, lat=None, radius=None, kwargs)


Synopsis:

Récupère tous les ports aux alentours d'un point géographique donné.

---

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
| lon | flottant | (minLon,maxLon) | ? | longitude du centre de la zone à requêter|
| lat | flottant | (minLat,maxLat) | ? | latitude du centre de la zone à requêter|
| radius | flottant | (0,?) | 100 ? |  rayon en kilomètres |


### Exemple : coordonnées de tous les ports recensés par Portic dans un rayon de 30km autour du port de La Rochelle 

In [7]:
from poitousprint import Portic
portic_client = Portic()

portic_client.get_departures_details(
 lat = 46.166667, # latitude du port de La Rochelle
 lon = -1.15, # longitude du port de La Rochelle
 radius = 30
)[0:1]

[{'departure': 'La Rochelle',
  'departure_uhgs_id': 'A0198999',
  'departure_latitude': '46.166667',
  'departure_longitude': '-1.15'}]

## portic_client.get_destinations_aggregated(lon=None, lat=None, radius=None, kwargs)

Synopsis:

Retourne une aggrégation des amirautés de départ de voyages qui arrivent dans des points situés dans le voisinage (voir paramètre radius) du point requêté.

'label': amirauté de départ

'value': nombre de voyages partis de cette amirauté

---

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
| lon | flottant | (minLon,maxLon) | ? | longitude du centre de la zone à requêter|
| lat | flottant | (minLat,maxLat) | ? | latitude du centre de la zone à requêter|
| radius | flottant | (0,?) | 100 ? |  rayon en kilomètres |


### Exemple : amirautés qui envoient des bateaux autour de l'ile de Ré (à 10km à la ronde), et nombre de bateaux envoyés

In [19]:
from poitousprint import Portic
portic_client = Portic()

portic_client.get_destinations_aggregated(
 lat = 46.2, # latitude du port de L'Ile de Ré
 lon = -1.416667, # longitude du port de L'Ile de Ré
 radius = 10
)[0:1]

[{'label': 'Barfleur', 'value': 1, 'id': 'Barfleur'},
 {'label': 'Bordeaux', 'value': 19, 'id': 'Bordeaux'},
 {'label': 'Boulogne', 'value': 1, 'id': 'Boulogne'},
 {'label': 'Brest', 'value': 8, 'id': 'Brest'},
 {'label': 'Cherbourg', 'value': 2, 'id': 'Cherbourg'},
 {'label': 'Coutances', 'value': 1, 'id': 'Coutances'},
 {'label': 'Dieppe', 'value': 1, 'id': 'Dieppe'},
 {'label': 'Dunkerque', 'value': 2, 'id': 'Dunkerque'},
 {'label': 'Eu et Tréport', 'value': 1, 'id': 'Eu et Tréport'},
 {'label': 'Granville', 'value': 3, 'id': 'Granville'},
 {'label': 'Havre', 'value': 3, 'id': 'Havre'},
 {'label': 'Honfleur', 'value': 2, 'id': 'Honfleur'},
 {'label': 'La Hougue', 'value': 2, 'id': 'La Hougue'},
 {'label': 'La Rochelle', 'value': 423, 'id': 'La Rochelle'},
 {'label': 'Marennes', 'value': 96, 'id': 'Marennes'},
 {'label': 'Morlaix', 'value': 6, 'id': 'Morlaix'},
 {'label': 'Nantes', 'value': 16, 'id': 'Nantes'},
 {'label': 'Quimper', 'value': 20, 'id': 'Quimper'},
 {'label': 'Rouen', 

## portic_client.get_ports(srid=None)

 Synopsis:
 Retourne une liste de *ports_points* au format JSON

---

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
|srid|int|---|---|identifiant de référence spatiale unique associé à un système de coordonnées, une tolérance et une résolution spécifiques|


### Exemple : description du port d'srid 4326 au format JSON

In [12]:
from poitousprint import Portic
portic_client = Portic()

resp = portic_client.get_ports(
    srid = 4326
)
print(json.dumps(resp[0:5], indent=2))

[
  {
    "ogc_fid": 11,
    "uhgs_id": "A1965015",
    "total": 13,
    "toponym": "Bourg [de l' Ile aux Moines]",
    "belonging_states": "[{\"1749-1815\" : \"France\"}]",
    "belonging_substates": null,
    "status": null,
    "geonameid": 6620139.0,
    "admiralty": "Vannes",
    "province": "Bretagne",
    "shiparea": "ACE-IROI",
    "point": "{\"type\":\"Point\",\"coordinates\":[-2.843156,47.598946]}"
  }
]


# Méthodes pour l'accès à TOFLIT
pouvant être appelées depuis `toflit_client` :

## toflit_client.get_flows(start_year=None, end_year=None, year=None, params=None, kwargs)

Synopsis : 
récupère les flux toflit18 tels que présentés au format CSV, avec possibilités de filtrage avancé.

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
| start_year | <int/string> | | None | année de début |
| end_year | <int/string> | | None | année de fin |
| year | <int/string> | | None | année exacte (l'emporte sur fin/début) |
| params | list |  | toutes les colonnes | propriétés à renvoyer |

\+ tous les noms de propriétés des flux pouvant être donnés en paramètres pour faire office de filtres (accepte une valeur unique, ou une liste de valeurs possibles pour que l'objet filtré soit valide)

## Exemple : tous les flows toflit de 1780 à 1790, enregistrés dans la direction des Fermes de La Rochelle, et où le produit commercé est du vin (on filtre le résultat sur certains params seulement)


In [2]:
from poitousprint import Toflit
toflit_client = Toflit()

flows = toflit_client.get_flows(
    start_year=1780, 
    end_year=1790, 
    customs_region='La Rochelle',
    product_revolutionempire='Vins divers',
    params=['customs_region', 'partner_simplification', 'product_revolutionempire', 'quantity', 'quantity_unit', 'customs_region_origin']
)
flows[0:1]

# util pour organiser un flow en ensembles thématiques (ne fonctionne que sur un flow avec params=all)
# flow = flows[0]
# nest_toflit18_flow(flow)

[{'customs_region': 'La Rochelle',
  'quantity': '25',
  'quantity_unit': 'tonneaux',
  'customs_region_origin': 'La Rochelle',
  'partner_simplification': 'Danemark',
  'product_revolutionempire': 'Vins divers'}]

## toflit_client.get_customs_regions()

Synopsis :

Récupère toutes les customs region (direction des Fermes) de la base

---

### Exemple

In [9]:
from poitousprint import Toflit
toflit_client = Toflit()


toflit_client.get_customs_regions()[0:1]

[{'id': 'Amiens', 'name': 'Amiens'}]

## toflit_client.get_product_classifications()

Synopsis :

Récupère les classifications de produits

---

### Exemple : on récupère la classification source (commenter la ligne resp['children']=None pour avoir les classifications filles)

In [2]:
from poitousprint import Toflit
toflit_client = Toflit()

resp = toflit_client.get_product_classifications()
# removing children for the sake of logging
# resp['children'] = None
print(json.dumps(resp, indent=2))

{
  "name": "Source",
  "description": "the product names as transcribed from archive volumes",
  "model": "product",
  "source": true,
  "id": "product_source",
  "slug": "source",
  "author": "TOFLIT 18",
  "parent": null,
  "groupsCount": 60193,
  "itemsCount": null,
  "unclassifiedItemsCount": 1,
  "completion": 0,
  "children": [
    {
      "name": "Orthographic normalization",
      "description": "The orthographic variations of product names are merged",
      "model": "product",
      "id": "product_orthographic",
      "slug": "orthographic",
      "author": "Pierre Gervais",
      "parent": "product_source",
      "groupsCount": 28598,
      "itemsCount": 60193,
      "unclassifiedItemsCount": 1,
      "completion": 99,
      "children": [
        {
          "name": "Simplification",
          "description": "Different product names designating the same good are merged",
          "model": "product",
          "id": "product_simplification",
          "slug": "simplificatio

## toflit_client.get_partner_classifications()

Synopsis :

Récupère les classifications de partenaires

---

### Exemple : on récupère toute l'arborescence de classifications (décommenter la ligne resp['children']=None pour n'avoir que la classification source)

In [19]:
from poitousprint import Toflit
toflit_client = Toflit()

resp = toflit_client.get_partner_classifications()
# removing children for the sake of logging
resp['children'] = None
print(json.dumps(resp, indent=2))

{
  "name": "Source",
  "description": "the partner names as transcribed from archive volumes",
  "model": "partner",
  "source": true,
  "id": "partner_source",
  "slug": "source",
  "author": "TOFLIT 18",
  "parent": null,
  "groupsCount": 1042,
  "itemsCount": null,
  "unclassifiedItemsCount": 1,
  "completion": 0,
  "children": null
}


## toflit_client.get_classification_groups(classification="product_simplification", kwargs)

Synopsis :

Récupère l'ensemble des catégories associées à une classification en particulier (sans le détail des valeurs)

---

### Exemple : récupération des 5 premières catégories de la classification partner grouping

In [19]:
from poitousprint import Toflit
toflit_client = Toflit()

toflit_client.get_classification_groups('partner_grouping')[0:5]

[{'id': '????~partner_grouping', 'name': '????'},
 {'id': 'Afrique~partner_grouping', 'name': 'Afrique'},
 {'id': 'Allemagne~partner_grouping', 'name': 'Allemagne'},
 {'id': 'Amériques~partner_grouping', 'name': 'Amériques'},
 {'id': 'Angleterre~partner_grouping', 'name': 'Angleterre'}]

## toflit_client.get_classification_search(classification='product_revolutionempire', kwargs)

Synopsis :

Récupère le détail des groupements associés à une classification en particulier.

---

Paramètre d'URL `classification` : le nom de la classification préfixé par son type (ex. "product_simplification", ou "partner_source")

---

### Exemple : produits sources recouverts par 2 catégories de product orthographic

In [ ]:
from poitousprint import Toflit
toflit_client = Toflit()

toflit_client.get_classification_search('product_orthographic')[0:2]

## toflit_client.get_locations(classification='partner_orthographic', kwargs)

Synopsis :

Récupère le réseau des lieux (directions et partenaires) et le montant de leurs échanges

---

Paramètre `classification` : l'id de la classification de partenaire à utiliser

---

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
| dateMin | entier | (années) | ? | année de début |
| dateMax | entier | (années) | ? | année de fin |
| kind | string | ('total', 'import', 'export') | 'total' | type de flux |
| sourceType | string | (types de source) | ? | id du type de source à utiliser |
| product | liste d'objets | (objects avec {{"id","name","value"}) | None | liste des produits à filtrer |
| productClassification | string | (ids de classification) | None | Classification de produit à utiliser pour le filtre |

---

### Exemple 

In [22]:
from poitousprint import Toflit
toflit_client = Toflit()

toflit_client.get_locations(
    classification = 'partner_grouping', 
    dateMin = 1750,
    dateMax = 1760,
    kind = 'total',
    sourceType = 'Best Guess national product x partner'
)[0:1]

[{'partner': 'Afrique',
  'region': 'France par la Compagnie des Indes',
  'count': 13,
  'value': 2524704.787475586},
 {'partner': 'Afrique',
  'region': 'Nantes',
  'count': 102,
  'value': 1794216.5125780106},
 {'partner': 'Afrique',
  'region': 'La Rochelle',
  'count': 71,
  'value': 742121.5014429092},
 {'partner': 'Afrique',
  'region': 'Rouen',
  'count': 86,
  'value': 398855.30004262924},
 {'partner': 'Afrique',
  'region': 'Rennes',
  'count': 47,
  'value': 136250.19985961914},
 {'partner': 'Afrique', 'region': 'Bordeaux', 'count': 54, 'value': 398664},
 {'partner': 'Asie',
  'region': 'France par la Compagnie des Indes',
  'count': 22,
  'value': 99432352.73144531},
 {'partner': 'Amériques',
  'region': 'France par la Compagnie des Indes',
  'count': 8,
  'value': 6700916.75},
 {'partner': 'Angleterre',
  'region': 'Rouen',
  'count': 165,
  'value': 3559982.7034116983},
 {'partner': 'Angleterre',
  'region': 'Bordeaux',
  'count': 51,
  'value': 8431979.040771484},
 {'par

## toflit_client.get_flows_per_year(type, kwargs)

Synopsis :

Récupère les flux par année par direction ou par type de source

---

Paramètre `type` : le type de flux 'direction' ou 'sourceType'

---

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
| dateMin | entier | (années) | ? | année de début |
| dateMax | entier | (années) | ? | année de fin |
| kind | string | ('total', 'import', 'export') | 'total' | type de flux |
| sourceType | string | (types de source) | ? | id du type de source à utiliser |
| partner | liste d'objets | (objects avec {"name","id"}) | None | les partenaires commerciaux à prendre en compte (e.g. {name: 'Alsace', id: 'Alsace~partner_orthographic'}) |
| partnerClassification | string | (ids de classification) | None | Classification de partenaire à utiliser pour le filtre |
| product | liste d'objets | (objects avec {{"id","name","value"}) | None | liste des produits à filtrer |
| productClassification | string | (ids de classification) | None | Classification de produit à utiliser pour le filtre |
| direction | chaîne | (noms de direction) | '$all$' | nom de la direction à filtrer |

---

### Exemple

In [ ]:
from poitousprint import Toflit
toflit_client = Toflit()

toflit_client.get_flows_per_year(
    type ='sourceType', 
    dateMin = 1750,
    dateMax = 1780,
    sourceType = 'National toutes directions partenaires manquants'
)

## toflit_client.get_time_series(kwargs)

Synopsis :

Récupère des séries temporelles à propos des flux de marchandises

---

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
| dateMin | entier | (années) | ? | année de début |
| dateMax | entier | (années) | ? | année de fin |
| kind | string | ('total', 'import', 'export') | 'total' | type de flux |
| sourceType | string | (types de source) | ? | id du type de source à utiliser |
| partner | liste d'objets | (objects avec {"name","id"}) | None | les partenaires commerciaux à prendre en compte (e.g. {name: 'Alsace', id: 'Alsace~partner_orthographic'}) |
| partnerClassification | string | (ids de classification) | None | Classification de partenaire à utiliser pour le filtre |
| product | liste d'objets | (objects avec {{"id","name","value"}) | None | liste des produits à filtrer |
| productClassification | string | (ids de classification) | None | Classification de produit à utiliser pour le filtre |
| direction | chaîne | (noms de direction) | '$all$' | nom de la direction à filtrer |

---

### Exemple

In [ ]:
toflit_client.get_time_series(
    direction = 'La_Rochelle'
)[0:2]

## toflit_client.get_sources_types()

Synopsis :

Récupère les types de sources disponibles

---

### Exemple

In [16]:
from poitousprint import Toflit
toflit_client = Toflit()

toflit_client.get_sources_types()

['1792-both semester',
 '1792-first semester',
 'Best Guess customs region product x partner',
 'Best Guess national customs region',
 'Best Guess national partner',
 'Best Guess national product',
 'Best Guess national product x partner',
 'Compagnie des Indes',
 'Local',
 'National partenaires manquants',
 'National toutes directions partenaires manquants',
 'National toutes directions sans produits',
 'National toutes directions tous partenaires',
 'Objet Général',
 'Résumé',
 'Tableau Général',
 'Tableau des quantités']

# Utils



## nest_toflit18_flow(flow)


Synopsis:
Cette fonction organise un flux toflit18 tel que reçu depuis les données en csv
  sous la forme d un ensemble de clés thématiques qui contiennent chacun les propriétés correspondantes.

---

### Exemple : 

In [1]:
from poitousprint import Toflit
toflit_client = Toflit()

flows = toflit_client.get_flows(
    year=1789, 
    product_orthographic='dentelle de soie'
) 

flow = flows[0] 
nest_toflit18_flow(flow)

NameError: name 'nest_toflit18_flow' is not defined

## nest_portic_pointcall(pointcall)


Synopsis:
Cette fonction organise un pointcall portic tel que reçu depuis les données en csv
sous la forme d un ensemble de clés thématiques qui contiennent chacun les propriétés correspondantes.

---

### Exemple : 

In [6]:
from poitousprint import Toflit
toflit_client = Toflit()

pointcalls = portic_client.get_pointcalls(
    year=1787
)

pointcall = pointcalls[0] 
nest_portic_pointcall(pointcall)

test commodity_purpose val: None
test commodity_purpose2 val: None
test commodity_purpose3 val: None
test commodity_purpose4 val: None


{'source': {'source_doc_id': '00169127',
  'source_text': 'ANF, G5-33/6822',
  'source_suite': 'G5',
  'source_component': 'ANF, G5-33',
  'source_number': 6822.0,
  'source_other': None,
  'source_main_port_uhgs_id': 'A0193167',
  'source_main_port_toponyme': 'Bastia',
  'source_subset': None,
  'navigo_status': 'FC-RF',
  'source_1787_available': False,
  'source_1789_available': False,
  'record_id': '00170322'},
 'pointcall_dates': {'pointcall_in_date': '1787>05>29!',
  'pointcall_out_date': None,
  'date_fixed': None,
  'pointcall_outdate_uncertainity': -1,
  'outdate_fixed': None,
  'indate_fixed': None,
  'pointcall_in_date2': '1787=05=29!',
  'pointcall_out_date2': None},
 'pointcall_localization': {'pkid': 50443,
  'pointcall': None,
  'pointcall_uhgs_id': 'H9999999',
  'toponyme_fr': 'pas mentionné',
  'toponyme_en': 'Not mentioned by the source',
  'latitude': None,
  'longitude': None,
  'pointcall_admiralty': None,
  'pointcall_province': None,
  'pointcall_states': None,


## build_cooccurence_graph(data, key_1, key_2, **kwargs)
    
Synopsis :
Cette fonction prend un ensemble de dict (data) et deux noms de clés présents dans chaque dict.
Elle renvoie un graphe networkx de coocurrence entre les dicts.

---

Paramètres de requête spécifiques (**kwargs) :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
| color_1 | code rgb (string?) |  | rgb(0, 255, 0) | couleur des noeuds de type 1 |
| color_2 | code rgb (string?) |  | rgb(255, 0, 0) | couleur des noeuds de type 2 |
| node_min_size | int |  | 1 | taille min des noeuds |
| node_min_size | int |  | 10 | taille max des noeuds |

### Exemple : 

In [12]:
from poitousprint import Toflit
from poitousprint import build_cooccurence_graph
from ipysigma import Sigma

toflit_client = Toflit()
flows = toflit_client.get_flows(
    start_year=1780, 
    end_year=1790, 
    customs_region='La Rochelle'
)

Graph = build_cooccurence_graph(flows, 'partner', 'product')
Sigma(Graph, start_layout=True)

Sigma(data={'nodes': [('partner_Allemagne', {'type': 'partner', 'name': 'Allemagne', 'color': 'rgb(0, 255, 0)'…

## get_online_csv(url)

Synopsis:
Cette fonction permet de récupérer le contenu d'un csv en ligne.
Pour les google spreadsheets: fichier > publier sur le web > format csv > copier le lien.
  
Bien mettre l'url sous forme de string dans les params

In [15]:
from poitousprint import get_online_csv


pointcalls_from_csv = get_online_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSTh5UY2pCjlPQz2WzHhqeKbAqUHzBpetRZbM9C4SzWK4chFZlIOHLU_zVpMtY8n6kNH1k4hIgpXiRE/pub?gid=0&single=true&output=csv')

pointcalls_from_csv[0:1]

[{'pkid': '92179',
  'record_id': '348660',
  'pointcall': 'Caen',
  'pointcall_uhgs_id': 'A0132409',
  'toponyme_fr': 'Caen',
  'toponyme_en': 'Caen',
  'latitude': '49.183333',
  'longitude': '-0.35',
  'pointcall_admiralty': 'Caen',
  'pointcall_province': 'Normandie',
  'pointcall_states': '[{"1749-1815" : "France"}]',
  'pointcall_substates': '',
  'pointcall_states_en': '[{"1749-1815" : "France"}]',
  'pointcall_substates_en': '',
  'state_1789_fr': 'France',
  'state_1789_en': 'France',
  'substate_1789_fr': '',
  'substate_1789_en': '',
  'source_1787_available': 'FALSE',
  'source_1789_available': 'FALSE',
  'pointcall_status': 'siège amirauté',
  'shiparea': 'MAN-WIGH',
  'pointcall_point': '0101000020110F000060A2004C3A06E3C0E85C5BA73A0E5841',
  'ferme_direction': '',
  'ferme_bureau': '',
  'ferme_bureau_uncertainty': '',
  'partner_balance_1789': '',
  'partner_balance_supp_1789': 'France',
  'partner_balance_1789_uncertainty': '',
  'partner_balance_supp_1789_uncertainty':

## get_pointcalls_commodity_purposes_as_toflit_product(pointcalls, product_classification="product_simplification")

Synopsis :
Cette fonction prend en entrée une liste de pointcalls et un nom de classification de produits ("product_simplification" par défaut).
Elle renvoie en sortie la liste des dict de pointcalls enrichis avec une propriété "commodity_purposes" qui ont les propriétés existantes de PORTIC pour commodity_purpose[2,3,4] + une propriété "commodity_as_toflit qui donne la valeur correspondante dans toflit18.

In [16]:
from poitousprint import Portic
from poitousprint import get_pointcalls_commodity_purposes_as_toflit_product

portic_client = Portic()

pointcalls = portic_client.get_pointcalls(
 year = 1789
)

with_toflit_products = get_pointcalls_commodity_purposes_as_toflit_product(pointcalls, product_classification='product_simplification')

with_toflit_products[0:1]

[{'pkid': 88712,
  'record_id': '00345003',
  'pointcall': None,
  'pointcall_uhgs_id': 'H4444444',
  'toponyme_fr': 'pas identifié',
  'toponyme_en': 'To be identified',
  'latitude': None,
  'longitude': None,
  'pointcall_admiralty': None,
  'pointcall_province': None,
  'pointcall_states': None,
  'pointcall_substates': None,
  'pointcall_states_en': None,
  'pointcall_substates_en': None,
  'state_1789_fr': None,
  'state_1789_en': None,
  'substate_1789_fr': None,
  'substate_1789_en': None,
  'source_1787_available': False,
  'source_1789_available': False,
  'pointcall_status': None,
  'shiparea': None,
  'pointcall_point': None,
  'ferme_direction': None,
  'ferme_bureau': None,
  'ferme_bureau_uncertainty': None,
  'partner_balance_1789': None,
  'partner_balance_supp_1789': None,
  'partner_balance_1789_uncertainty': None,
  'partner_balance_supp_1789_uncertainty': None,
  'pointcall_out_date': None,
  'pointcall_action': 'In',
  'outdate_fixed': None,
  'pointcall_in_date':

## get_pointcalls_port_as_toflit_partner(pointcalls, product_classification="partner_simplification")

Synopsis :
 Cette fonction prend en entrée une liste de pointcalls et un nom de classification de partenaire ("partner_simplification" par défaut).
Elle renvoie en sortie la liste des dict de pointcalls enrichis avec une propriété "pointcall_as_toflit_partner".

In [21]:
from poitousprint import Portic
from poitousprint import get_pointcalls_port_as_toflit_partner

portic_client = Portic()

pointcalls = portic_client.get_pointcalls(
 year = 1789
)

get_pointcalls_port_as_toflit_partner(pointcalls, 'partner_grouping')[0:1]

[{'pkid': 88712,
  'record_id': '00345003',
  'pointcall': None,
  'pointcall_uhgs_id': 'H4444444',
  'toponyme_fr': 'pas identifié',
  'toponyme_en': 'To be identified',
  'latitude': None,
  'longitude': None,
  'pointcall_admiralty': None,
  'pointcall_province': None,
  'pointcall_states': None,
  'pointcall_substates': None,
  'pointcall_states_en': None,
  'pointcall_substates_en': None,
  'state_1789_fr': None,
  'state_1789_en': None,
  'substate_1789_fr': None,
  'substate_1789_en': None,
  'source_1787_available': False,
  'source_1789_available': False,
  'pointcall_status': None,
  'shiparea': None,
  'pointcall_point': None,
  'ferme_direction': None,
  'ferme_bureau': None,
  'ferme_bureau_uncertainty': None,
  'partner_balance_1789': None,
  'partner_balance_supp_1789': None,
  'partner_balance_1789_uncertainty': None,
  'partner_balance_supp_1789_uncertainty': None,
  'pointcall_out_date': None,
  'pointcall_action': 'In',
  'outdate_fixed': None,
  'pointcall_in_date':

## get_pointcalls_homeport_as_toflit_partner(pointcalls, product_classification="partner_simplification")
  
Synopsis :
Cette fonction prend en entrée une liste de pointcalls et un nom de classification de partenaire ("partner_simplification" par défaut).
Elle renvoie en sortie la liste des dict de pointcalls enrichis avec une propriété "homeport_as_toflit_partner".

In [20]:
from poitousprint import Portic
from poitousprint import get_pointcalls_homeport_as_toflit_partner

portic_client = Portic()

pointcalls = portic_client.get_pointcalls(
 year = 1789
)

get_pointcalls_homeport_as_toflit_partner(pointcalls, 'partner_simplification')[0:1]

[{'pkid': 88712,
  'record_id': '00345003',
  'pointcall': None,
  'pointcall_uhgs_id': 'H4444444',
  'toponyme_fr': 'pas identifié',
  'toponyme_en': 'To be identified',
  'latitude': None,
  'longitude': None,
  'pointcall_admiralty': None,
  'pointcall_province': None,
  'pointcall_states': None,
  'pointcall_substates': None,
  'pointcall_states_en': None,
  'pointcall_substates_en': None,
  'state_1789_fr': None,
  'state_1789_en': None,
  'substate_1789_fr': None,
  'substate_1789_en': None,
  'source_1787_available': False,
  'source_1789_available': False,
  'pointcall_status': None,
  'shiparea': None,
  'pointcall_point': None,
  'ferme_direction': None,
  'ferme_bureau': None,
  'ferme_bureau_uncertainty': None,
  'partner_balance_1789': None,
  'partner_balance_supp_1789': None,
  'partner_balance_1789_uncertainty': None,
  'partner_balance_supp_1789_uncertainty': None,
  'pointcall_out_date': None,
  'pointcall_action': 'In',
  'outdate_fixed': None,
  'pointcall_in_date':